In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
train = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv"))

In [3]:
test = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv"))

In [4]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
train['Region_Code'] = train['Region_Code'].astype('int16')
train['Policy_Sales_Channel'] = train['Policy_Sales_Channel'].astype('int16')

In [6]:
df_1 = train[train['Response'] == 1].sample(frac=0.2, random_state=0)
df_0 = train[train['Response'] == 0].sample(frac=0.1, random_state=0)
df = pd.concat([df_1, df_0], axis=0)
print(df_1.describe())
print(df_0.describe())

                 id            Age  Driving_License    Region_Code  \
count  2.830120e+05  283012.000000    283012.000000  283012.000000   
mean   5.755760e+06      43.277013         0.999060      26.840074   
std    3.312113e+06      11.484338         0.030643      11.296856   
min    8.200000e+01      20.000000         0.000000       0.000000   
25%    2.897558e+06      35.000000         1.000000      26.000000   
50%    5.752946e+06      43.000000         1.000000      28.000000   
75%    8.617492e+06      50.000000         1.000000      29.000000   
max    1.150480e+07      81.000000         1.000000      52.000000   

       Previously_Insured  Annual_Premium  Policy_Sales_Channel  \
count       283012.000000   283012.000000         283012.000000   
mean             0.002406    31877.109070             90.409495   
std              0.048995    17505.582904             55.913322   
min              0.000000     2630.000000              1.000000   
25%              0.000000    25963

In [7]:
X = df.drop(['Response', 'id'], axis=1)
y = df['Response']

In [8]:
X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
10628253,Male,39,1,28,0,1-2 Year,Yes,24224.0,26,214
1880466,Male,42,1,28,0,1-2 Year,Yes,44012.0,124,222
5477618,Female,47,1,28,0,1-2 Year,Yes,53511.0,26,115
10390038,Female,38,1,28,0,1-2 Year,Yes,29488.0,26,144
1643786,Male,37,1,7,0,1-2 Year,Yes,30140.0,124,81


In [9]:
cat_features = [0, 2, 3, 4, 5, 6, 8]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=22)

In [12]:
import gc
# 가비지 컬렉션 수행 전, 객체 수 확인
print("Before GC:", len(gc.get_objects()))

# 가비지 컬렉션 수행
gc.collect()

# 가비지 컬렉션 수행 후, 객체 수 확인
print("After GC:", len(gc.get_objects()))

Before GC: 187106
After GC: 186713


In [13]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=50,
    learning_rate=0.4,
    depth=8,
    one_hot_max_size=3,
    custom_loss=['AUC', 'Recall']
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

0:	learn: 0.5321572	test: 0.5319849	best: 0.5319849 (0)	total: 412ms	remaining: 20.2s
1:	learn: 0.4598108	test: 0.4596328	best: 0.4596328 (1)	total: 778ms	remaining: 18.7s
2:	learn: 0.4201709	test: 0.4199846	best: 0.4199846 (2)	total: 1.09s	remaining: 17.1s
3:	learn: 0.3959803	test: 0.3958022	best: 0.3958022 (3)	total: 1.42s	remaining: 16.3s
4:	learn: 0.3807773	test: 0.3806194	best: 0.3806194 (4)	total: 1.7s	remaining: 15.3s
5:	learn: 0.3711924	test: 0.3710815	best: 0.3710815 (5)	total: 1.99s	remaining: 14.6s
6:	learn: 0.3641345	test: 0.3640478	best: 0.3640478 (6)	total: 2.31s	remaining: 14.2s
7:	learn: 0.3577877	test: 0.3576681	best: 0.3576681 (7)	total: 2.63s	remaining: 13.8s
8:	learn: 0.3541714	test: 0.3541426	best: 0.3541426 (8)	total: 2.94s	remaining: 13.4s
9:	learn: 0.3523343	test: 0.3522134	best: 0.3522134 (9)	total: 3.24s	remaining: 13s
10:	learn: 0.3511824	test: 0.3511096	best: 0.3511096 (10)	total: 3.59s	remaining: 12.7s
11:	learn: 0.3499925	test: 0.3499535	best: 0.3499535 (1

In [14]:
y_pred = clf.predict(data=X_val)

In [15]:
def calc_accuracy(y_pred, y_val):
    acc = y_val == y_pred
    i = 0
    for d in acc:
        if d == True:
            i+=1
    s, n = 0, 0
    for j in y_pred:
        n+=1
        if j==1:
            s += 1
    print(i/acc.size, s, n)

In [16]:
calc_accuracy(y_pred, y_val)

0.813086788597435 21082 129199


In [17]:
X_all = train.drop(['Response', 'id'], axis=1)
y_pred_all = clf.predict(data=X_all)

In [18]:
#please execute once
y_all = train['Response']

In [19]:
calc_accuracy(y_pred_all, y_all)

0.857476680598825 1487647 11504798
